# 在mmdet框架中使用config配置文件构建网络模型

In [65]:
from   mmdet.models.builder import build_detector
from   mmcv import Config
import torch
import mmcv

In [89]:
from   os.path import join,realpath,relpath
import glob
config_dpath = 'configs'+"/yolo"
config_fpaths = list(glob.glob(join(config_dpath,'*.py')))
config_fpaths = [p for p in config_fpaths if p.find('_base_') == -1]
config_names = [relpath(p, config_dpath) for p in config_fpaths]

In [67]:
config_names

['yolov3 - 副本.py',
 'yolov3.py',
 'yolov3_d53_320_273e_coco.py',
 'yolov3_d53_mstrain-416_273e_coco.py',
 'yolov3_d53_mstrain-608_273e_coco.py']

### 1. 解析yolov3配置文件

In [90]:
for config_fname in config_names[1:2]:
    config_fpath = join(config_dpath, config_fname)
    config_mod = Config.fromfile(config_fpath)
print(config_fname,'\n',config_fpath,'\n',config_mod)

yolov3.py 
 configs/yolo\yolov3.py 
 Config (path: configs/yolo\yolov3.py): {'model': {'type': 'YOLOV3', 'pretrained': 'open-mmlab://darknet53', 'backbone': {'type': 'Darknet', 'depth': 53, 'out_indices': (3, 4, 5)}, 'neck': {'type': 'YOLOV3Neck', 'num_scales': 3, 'in_channels': [1024, 512, 256], 'out_channels': [512, 256, 128]}, 'bbox_head': {'type': 'YOLOV3Head', 'num_classes': 80, 'in_channels': [512, 256, 128], 'out_channels': [1024, 512, 256], 'anchor_generator': {'type': 'YOLOAnchorGenerator', 'base_sizes': [[(116, 90), (156, 198), (373, 326)], [(30, 61), (62, 45), (59, 119)], [(10, 13), (16, 30), (33, 23)]], 'strides': [32, 16, 8]}, 'bbox_coder': {'type': 'YOLOBBoxCoder'}, 'featmap_strides': [32, 16, 8], 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}, 'loss_conf': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}, 'loss_xy': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weigh

In [55]:
for k,v in  config_mod.items():
    print(k,":",v)

model : {'type': 'YOLOV3', 'pretrained': 'open-mmlab://darknet53', 'backbone': {'type': 'Darknet', 'depth': 53, 'out_indices': (3, 4, 5)}, 'neck': {'type': 'YOLOV3Neck', 'num_scales': 3, 'in_channels': [1024, 512, 256], 'out_channels': [512, 256, 128]}, 'bbox_head': {'type': 'YOLOV3Head', 'num_classes': 80, 'in_channels': [512, 256, 128], 'out_channels': [1024, 512, 256], 'anchor_generator': {'type': 'YOLOAnchorGenerator', 'base_sizes': [[(116, 90), (156, 198), (373, 326)], [(30, 61), (62, 45), (59, 119)], [(10, 13), (16, 30), (33, 23)]], 'strides': [32, 16, 8]}, 'bbox_coder': {'type': 'YOLOBBoxCoder'}, 'featmap_strides': [32, 16, 8], 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}, 'loss_conf': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}, 'loss_xy': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 2.0, 'reduction': 'sum'}, 'loss_wh': {'type': 'MSELoss', 'loss_weight': 2.

In [19]:
config_mod.model

{'type': 'YOLOV3',
 'pretrained': 'open-mmlab://darknet53',
 'backbone': {'type': 'Darknet', 'depth': 53, 'out_indices': (3, 4, 5)},
 'neck': {'type': 'YOLOV3Neck',
  'num_scales': 3,
  'in_channels': [1024, 512, 256],
  'out_channels': [512, 256, 128]},
 'bbox_head': {'type': 'YOLOV3Head',
  'num_classes': 80,
  'in_channels': [512, 256, 128],
  'out_channels': [1024, 512, 256],
  'anchor_generator': {'type': 'YOLOAnchorGenerator',
   'base_sizes': [[(116, 90), (156, 198), (373, 326)],
    [(30, 61), (62, 45), (59, 119)],
    [(10, 13), (16, 30), (33, 23)]],
   'strides': [32, 16, 8]},
  'bbox_coder': {'type': 'YOLOBBoxCoder'},
  'featmap_strides': [32, 16, 8],
  'loss_cls': {'type': 'CrossEntropyLoss',
   'use_sigmoid': True,
   'loss_weight': 1.0,
   'reduction': 'sum'},
  'loss_conf': {'type': 'CrossEntropyLoss',
   'use_sigmoid': True,
   'loss_weight': 1.0,
   'reduction': 'sum'},
  'loss_xy': {'type': 'CrossEntropyLoss',
   'use_sigmoid': True,
   'loss_weight': 2.0,
   'reducti

In [20]:
if 'pretrained' in config_mod.model:
    config_mod.model['pretrained'] = None

### 2.构建检测器（使用构建函数和hook）

In [23]:
from mmdet.models import build_detector
model=build_detector(config_mod.model)

In [24]:
print(model)

YOLOV3(
  (backbone): Darknet(
    (conv1): ConvModule(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (conv_res_block1): Sequential(
      (conv): ConvModule(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (res0): ResBlock(
        (conv1): ConvModule(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activate): LeakyReLU(negative_slope=0.1, inplace=True)
        )
        (conv2): ConvModule(
          (conv): Conv2d(32, 64, kerne

In [25]:
from mmdet.models import DETECTORS
DETECTORS.module_dict

{'SingleStageDetector': mmdet.models.detectors.single_stage.SingleStageDetector,
 'ATSS': mmdet.models.detectors.atss.ATSS,
 'TwoStageDetector': mmdet.models.detectors.two_stage.TwoStageDetector,
 'CascadeRCNN': mmdet.models.detectors.cascade_rcnn.CascadeRCNN,
 'CornerNet': mmdet.models.detectors.cornernet.CornerNet,
 'DETR': mmdet.models.detectors.detr.DETR,
 'FastRCNN': mmdet.models.detectors.fast_rcnn.FastRCNN,
 'FasterRCNN': mmdet.models.detectors.faster_rcnn.FasterRCNN,
 'FCOS': mmdet.models.detectors.fcos.FCOS,
 'FOVEA': mmdet.models.detectors.fovea.FOVEA,
 'FSAF': mmdet.models.detectors.fsaf.FSAF,
 'GFL': mmdet.models.detectors.gfl.GFL,
 'GridRCNN': mmdet.models.detectors.grid_rcnn.GridRCNN,
 'HybridTaskCascade': mmdet.models.detectors.htc.HybridTaskCascade,
 'MaskRCNN': mmdet.models.detectors.mask_rcnn.MaskRCNN,
 'MaskScoringRCNN': mmdet.models.detectors.mask_scoring_rcnn.MaskScoringRCNN,
 'NASFCOS': mmdet.models.detectors.nasfcos.NASFCOS,
 'PAA': mmdet.models.detectors.paa.PAA

In [26]:
# model=build_detector({'type':'YOLOV5','backbone':{}, 'neck':{},'bbox_head':{}})

In [27]:
m=DETECTORS.get(config_mod.model['type'])
# print(m(backbone={},neck={},bbox_head={}))

### 2.1 构建主干网络

In [28]:
from mmdet.models import build_backbone,BACKBONES
BACKBONES.module_dict

{'Darknet': mmdet.models.backbones.darknet.Darknet,
 'ResNet': mmdet.models.backbones.resnet.ResNet,
 'ResNetV1d': mmdet.models.backbones.resnet.ResNetV1d,
 'DetectoRS_ResNet': mmdet.models.backbones.detectors_resnet.DetectoRS_ResNet,
 'DetectoRS_ResNeXt': mmdet.models.backbones.detectors_resnext.DetectoRS_ResNeXt,
 'HourglassNet': mmdet.models.backbones.hourglass.HourglassNet,
 'HRNet': mmdet.models.backbones.hrnet.HRNet,
 'ResNeXt': mmdet.models.backbones.resnext.ResNeXt,
 'RegNet': mmdet.models.backbones.regnet.RegNet,
 'Res2Net': mmdet.models.backbones.res2net.Res2Net,
 'ResNeSt': mmdet.models.backbones.resnest.ResNeSt,
 'SSDVGG': mmdet.models.backbones.ssd_vgg.SSDVGG,
 'TridentResNet': mmdet.models.backbones.trident_resnet.TridentResNet}

2.1.1 Use  backbone  builder

In [29]:
backbone_config=config_mod.model['backbone']
print(backbone_config)

{'type': 'Darknet', 'depth': 53, 'out_indices': (3, 4, 5)}


In [30]:
backone=build_backbone(config_mod.model['backbone'])
print(backone)

Darknet(
  (conv1): ConvModule(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activate): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv_res_block1): Sequential(
    (conv): ConvModule(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (res0): ResBlock(
      (conv1): ConvModule(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv2): ConvModule(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  

2.1.2 Use code hook

In [31]:
bo=BACKBONES.get(backbone_config['type'])
bc=backbone_config.pop('type')
print(bc)

Darknet


In [32]:
backbone_config

{'depth': 53, 'out_indices': (3, 4, 5)}

In [33]:
bo(**backbone_config)

Darknet(
  (conv1): ConvModule(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activate): LeakyReLU(negative_slope=0.1, inplace=True)
  )
  (conv_res_block1): Sequential(
    (conv): ConvModule(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (res0): ResBlock(
      (conv1): ConvModule(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activate): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (conv2): ConvModule(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  

### 2.2 Builder  necks 

In [34]:
from mmdet.models import build_neck,NECKS
NECKS.module_dict

{'BFP': mmdet.models.necks.bfp.BFP,
 'ChannelMapper': mmdet.models.necks.channel_mapper.ChannelMapper,
 'FPG': mmdet.models.necks.fpg.FPG,
 'FPN': mmdet.models.necks.fpn.FPN,
 'FPN_CARAFE': mmdet.models.necks.fpn_carafe.FPN_CARAFE,
 'HRFPN': mmdet.models.necks.hrfpn.HRFPN,
 'NASFPN': mmdet.models.necks.nas_fpn.NASFPN,
 'NASFCOS_FPN': mmdet.models.necks.nasfcos_fpn.NASFCOS_FPN,
 'PAFPN': mmdet.models.necks.pafpn.PAFPN,
 'RFP': mmdet.models.necks.rfp.RFP,
 'YOLOV3Neck': mmdet.models.necks.yolo_neck.YOLOV3Neck}

In [39]:
neck = build_neck(config_mod.model['neck'])

In [40]:
neck

YOLOV3Neck(
  (detect1): DetectionBlock(
    (conv1): ConvModule(
      (conv): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (conv2): ConvModule(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (conv3): ConvModule(
      (conv): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (conv4): ConvModule(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(102

In [43]:
neck_config=config_mod.model['neck']
neck_config.pop('type')
print(neck_config)

{'num_scales': 3, 'in_channels': [1024, 512, 256], 'out_channels': [512, 256, 128]}


In [44]:
NECKS.get('YOLOV3Neck')(**neck_config)

YOLOV3Neck(
  (detect1): DetectionBlock(
    (conv1): ConvModule(
      (conv): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (conv2): ConvModule(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (conv3): ConvModule(
      (conv): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (conv4): ConvModule(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(102

#### 2.3 构建检测头 head

In [45]:
from  mmdet.models import build_head,HEADS
HEADS.module_dict

{'AnchorFreeHead': mmdet.models.dense_heads.anchor_free_head.AnchorFreeHead,
 'AnchorHead': mmdet.models.dense_heads.anchor_head.AnchorHead,
 'ATSSHead': mmdet.models.dense_heads.atss_head.ATSSHead,
 'RPNHead': mmdet.models.dense_heads.rpn_head.RPNHead,
 'StageCascadeRPNHead': mmdet.models.dense_heads.cascade_rpn_head.StageCascadeRPNHead,
 'CascadeRPNHead': mmdet.models.dense_heads.cascade_rpn_head.CascadeRPNHead,
 'CornerHead': mmdet.models.dense_heads.corner_head.CornerHead,
 'CentripetalHead': mmdet.models.dense_heads.centripetal_head.CentripetalHead,
 'EmbeddingRPNHead': mmdet.models.dense_heads.embedding_rpn_head.EmbeddingRPNHead,
 'FCOSHead': mmdet.models.dense_heads.fcos_head.FCOSHead,
 'FoveaHead': mmdet.models.dense_heads.fovea_head.FoveaHead,
 'RetinaHead': mmdet.models.dense_heads.retina_head.RetinaHead,
 'FreeAnchorRetinaHead': mmdet.models.dense_heads.free_anchor_retina_head.FreeAnchorRetinaHead,
 'FSAFHead': mmdet.models.dense_heads.fsaf_head.FSAFHead,
 'GuidedAnchorHead'

In [47]:
config_mod.model['bbox_head']

{'type': 'YOLOV3Head',
 'num_classes': 80,
 'in_channels': [512, 256, 128],
 'out_channels': [1024, 512, 256],
 'anchor_generator': {'type': 'YOLOAnchorGenerator',
  'base_sizes': [[(116, 90), (156, 198), (373, 326)],
   [(30, 61), (62, 45), (59, 119)],
   [(10, 13), (16, 30), (33, 23)]],
  'strides': [32, 16, 8]},
 'bbox_coder': {'type': 'YOLOBBoxCoder'},
 'featmap_strides': [32, 16, 8],
 'loss_cls': {'type': 'CrossEntropyLoss',
  'use_sigmoid': True,
  'loss_weight': 1.0,
  'reduction': 'sum'},
 'loss_conf': {'type': 'CrossEntropyLoss',
  'use_sigmoid': True,
  'loss_weight': 1.0,
  'reduction': 'sum'},
 'loss_xy': {'type': 'CrossEntropyLoss',
  'use_sigmoid': True,
  'loss_weight': 2.0,
  'reduction': 'sum'},
 'loss_wh': {'type': 'MSELoss', 'loss_weight': 2.0, 'reduction': 'sum'},
 'train_cfg': {'assigner': {'type': 'GridAssigner',
   'pos_iou_thr': 0.5,
   'neg_iou_thr': 0.5,
   'min_pos_iou': 0}},
 'test_cfg': {'nms_pre': 1000,
  'min_bbox_size': 0,
  'score_thr': 0.05,
  'conf_th

In [49]:
bbox_head=build_head(config_mod.model['bbox_head'])
print(bbox_head)

YOLOV3Head(
  (loss_cls): CrossEntropyLoss()
  (loss_conf): CrossEntropyLoss()
  (loss_xy): CrossEntropyLoss()
  (loss_wh): MSELoss()
  (convs_bridge): ModuleList(
    (0): ConvModule(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): ConvModule(
      (conv): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): ConvModule(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
  )
  

In [50]:
head=HEADS.get('YOLOV3Head')

In [51]:
head_config = config_mod.model['bbox_head']
print(head_config)

{'type': 'YOLOV3Head', 'num_classes': 80, 'in_channels': [512, 256, 128], 'out_channels': [1024, 512, 256], 'anchor_generator': {'type': 'YOLOAnchorGenerator', 'base_sizes': [[(116, 90), (156, 198), (373, 326)], [(30, 61), (62, 45), (59, 119)], [(10, 13), (16, 30), (33, 23)]], 'strides': [32, 16, 8]}, 'bbox_coder': {'type': 'YOLOBBoxCoder'}, 'featmap_strides': [32, 16, 8], 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}, 'loss_conf': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0, 'reduction': 'sum'}, 'loss_xy': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 2.0, 'reduction': 'sum'}, 'loss_wh': {'type': 'MSELoss', 'loss_weight': 2.0, 'reduction': 'sum'}, 'train_cfg': {'assigner': {'type': 'GridAssigner', 'pos_iou_thr': 0.5, 'neg_iou_thr': 0.5, 'min_pos_iou': 0}}, 'test_cfg': {'nms_pre': 1000, 'min_bbox_size': 0, 'score_thr': 0.05, 'conf_thr': 0.005, 'nms': {'type': 'nms', 'iou_threshold': 0.45

In [52]:
head_config.pop('type')

'YOLOV3Head'

In [53]:
head(**head_config)

YOLOV3Head(
  (loss_cls): CrossEntropyLoss()
  (loss_conf): CrossEntropyLoss()
  (loss_xy): CrossEntropyLoss()
  (loss_wh): MSELoss()
  (convs_bridge): ModuleList(
    (0): ConvModule(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): ConvModule(
      (conv): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): ConvModule(
      (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): LeakyReLU(negative_slope=0.1, inplace=True)
    )
  )
  

### 3. 构建数据集 

In [63]:
from mmdet.datasets import build_dataset,build_dataloader,DATASETS,PIPELINES
DATASETS.module_dict

{'CustomDataset': mmdet.datasets.custom.CustomDataset,
 'CocoDataset': mmdet.datasets.coco.CocoDataset,
 'CityscapesDataset': mmdet.datasets.cityscapes.CityscapesDataset,
 'ConcatDataset': mmdet.datasets.dataset_wrappers.ConcatDataset,
 'RepeatDataset': mmdet.datasets.dataset_wrappers.RepeatDataset,
 'ClassBalancedDataset': mmdet.datasets.dataset_wrappers.ClassBalancedDataset,
 'DeepFashionDataset': mmdet.datasets.deepfashion.DeepFashionDataset,
 'LVISV05Dataset': mmdet.datasets.lvis.LVISV05Dataset,
 'LVISDataset': mmdet.datasets.lvis.LVISV05Dataset,
 'LVISV1Dataset': mmdet.datasets.lvis.LVISV1Dataset,
 'XMLDataset': mmdet.datasets.xml_style.XMLDataset,
 'VOCDataset': mmdet.datasets.voc.VOCDataset,
 'WIDERFaceDataset': mmdet.datasets.wider_face.WIDERFaceDataset}

In [59]:
PIPELINES.module_dict

{'Compose': mmdet.datasets.pipelines.compose.Compose,
 'AutoAugment': mmdet.datasets.pipelines.auto_augment.AutoAugment,
 'Shear': mmdet.datasets.pipelines.auto_augment.Shear,
 'Rotate': mmdet.datasets.pipelines.auto_augment.Rotate,
 'Translate': mmdet.datasets.pipelines.auto_augment.Translate,
 'ColorTransform': mmdet.datasets.pipelines.auto_augment.ColorTransform,
 'EqualizeTransform': mmdet.datasets.pipelines.auto_augment.EqualizeTransform,
 'BrightnessTransform': mmdet.datasets.pipelines.auto_augment.BrightnessTransform,
 'ContrastTransform': mmdet.datasets.pipelines.auto_augment.ContrastTransform,
 'ToTensor': mmdet.datasets.pipelines.formating.ToTensor,
 'ImageToTensor': mmdet.datasets.pipelines.formating.ImageToTensor,
 'Transpose': mmdet.datasets.pipelines.formating.Transpose,
 'ToDataContainer': mmdet.datasets.pipelines.formating.ToDataContainer,
 'DefaultFormatBundle': mmdet.datasets.pipelines.formating.DefaultFormatBundle,
 'Collect': mmdet.datasets.pipelines.formating.Colle

In [91]:
config_mod.data['train']

{'type': 'CocoDataset',
 'ann_file': 'F:/datasets/COCO2017/annotations/instances_train2017.json',
 'img_prefix': 'F:/datasets/COCO2017/train2017/',
 'pipeline': [{'type': 'LoadImageFromFile', 'to_float32': True},
  {'type': 'LoadAnnotations', 'with_bbox': True},
  {'type': 'PhotoMetricDistortion'},
  {'type': 'Resize',
   'img_scale': [(320, 320), (608, 608)],
   'keep_ratio': True}]}

In [92]:
cocodataset=build_dataset(config_mod.data['train'])

loading annotations into memory...
Done (t=42.56s)
creating index...
index created!


In [93]:
dataloader=build_dataloader(cocodataset,1,1)

In [97]:
for batch in dataloader:
    print(batch)
    break